<a href="https://colab.research.google.com/github/RubenUnitel/Clase/blob/main/Practica_Banco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Descargar la Base de Datos

In [38]:
!gdown 1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX

Downloading...
From: https://drive.google.com/uc?id=1puf9UBuzaVqMW3ZMMXhd626E5yVJd_QX
To: /content/data_berka.db
100% 70.2M/70.2M [00:00<00:00, 275MB/s]


1.1) Cargamos la extensión sql (nos permite ejecutar comandos sql directamente en colab)

In [39]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


1.2) Conectar a la Base de Datos

In [40]:
%sql sqlite:///data_berka.db

1.3) Comprobamos el esquema de la BBDD
Para ejecutar comandos sql, se debe añadir:
 "%sql comando"
 o en múltiples líneas:
"%%sql
comando_línea1
comando_línea2"

In [41]:
%%sql
SELECT name, type
FROM sqlite_master
WHERE type = "table"

 * sqlite:///data_berka.db
Done.


name,type
DISTRICT,table
ACCOUNT,table
TRANS,table
CLIENT,table
DISPOSITION,table
CREDIT_CARD,table
LOAN,table
PERMANENT_ORDER,table


# 2) Ejercicios

2.1) Muestre los datos de la tabla cliente

In [42]:
%sql SELECT * FROM CLIENT LIMIT 5

 * sqlite:///data_berka.db
Done.


client_id,birth_number,district_id
1,706213,18
2,450204,1
3,406009,1
4,561201,5
5,605703,5


2.2) Liste todos los clientes con cuentas activas y préstamos, incluyendo dinero prestado y saldos de cuenta

In [43]:
%%sql SELECT
  c.client_id AS Cliente,
  ROUND(SUM(l.amount), 2) AS Importe_Prestado,
  ROUND(SUM(CASE
      WHEN t.type = 'PRIJEM' THEN t.amount
      WHEN t.type = 'VYDAJ' THEN -t.amount
      ELSE 0
    END), 2) AS Balance_Cuenta
FROM CLIENT c
LEFT JOIN DISPOSITION d ON d.client_id = c.client_id
LEFT JOIN ACCOUNT a ON a.account_id = d.account_id
LEFT JOIN LOAN l ON l.account_id = a.account_id
LEFT JOIN TRANS t ON t.account_id = a.account_id
WHERE d.type = 'OWNER'
GROUP BY c.client_id
LIMIT 5

 * sqlite:///data_berka.db
Done.


Cliente,Importe_Prestado,Balance_Cuenta
1,38695056.0,273536.2
4,None,51096.2
6,None,33720.6
7,None,28088.6
8,None,47667.9


2.3) Identifique clientes con múltiples cuentas.

In [44]:
%%sql
SELECT
  c.client_id AS Cliente,
  COUNT(a.account_id) AS Total_Cuentas
FROM client c
LEFT JOIN disposition d ON d.client_id = c.client_id
LEFT JOIN account a ON a.account_id = d.account_id
GROUP BY c.client_id
HAVING Total_Cuentas > 1

 * sqlite:///data_berka.db
Done.


Cliente,Total_Cuentas
1,2


In [45]:
#%%sql UPDATE disposition set client_id = 1 WHERE disp_id = 2

2.4) Recupere transacciones del año 1998, ordenadas por cantidad de transacción, e identifique las 10 transacciones más grandes.